In [1]:
"""Load the primitive model and the DSL parse"""
from karanir.thanagor.model import *
from karanir.thanagor import KaranirThanagor, config

[Taichi] version 1.7.0, llvm 15.0.7, commit 7b58b0ff, osx, python 3.9.13


[I 12/29/23 14:34:43.670 4911156] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
"""Define the domain specification for the demo test case"""
from karanir.thanagor.domain import load_domain_string
demo_domain_string = f"""
(domain scourge_demo)
(:type
    object - vector[float,100]
    position - vector[float,2]
    color - vector[float, 64]
    category
)
(:predicate
    is-red ?x-object -> boolean
    is-blue ?x-object -> boolean
    is-ship ?x-object -> boolean
    is-house ?x-object -> boolean
)
(:derived
    (
        is-good ?x
        expr: ( or (is-red ?x) (is-house ?x))
    )
    (
        is-cool ?x
        expr: (or (is-good ?x) (is-blue ?x))
    )
)
(:constraint
    (color: is-red is-blue)
    (category: is-ship is-house)
)
(:action
    (
        name:pick-up
        parameters:?x-object ?y
        precondition: (is-red ?x)
        effect: (
            if (is-blue ?y)
                (assign (is-blue ?x) true)
                (assign (is-red ?y) false)
            )
    )
)
"""
domain = load_domain_string(demo_domain_string)
domain.print_summary()

domain:
  scourge_demo
types:
  object - vector[float,['100']]
  position - vector[float,['2']]
  color - vector[float,['64']]
  category - object
predicates:
  is-red:['?x-object'] -> boolean
  is-blue:['?x-object'] -> boolean
  is-ship:['?x-object'] -> boolean
  is-house:['?x-object'] -> boolean
actions:
 name:pick-up
  params:['?x-object', '?y']
  precond:(is-red ?x)
  effect:(if (is-blue ?y) (assign (is-blue ?x) (true)) (assign (is-red ?y) (false)))


In [3]:
model = KaranirThanagor(domain, config)

In [4]:
p = "(is-cool ($0))"

obj_dim = config.object_dim
context = {
    "end": torch.ones([1]),
    "features": torch.randn([1,obj_dim]),
    "executor": model.central_executor
}

output = model.central_executor.evaluate(p, {0:context})

print(output["end"])

o0 = model.central_executor.evaluate("(is-good $0)", {0:context})
print(o0["end"])
o1 = model.central_executor.evaluate("(is-red $0)", {0:context})
o2 = model.central_executor.evaluate("(is-house $0)", {0:context})
print(o1["end"],o2["end"])
o3 = model.central_executor.evaluate("(is-blue $0)", {0:context})
print(o3["end"])

o3 = model.central_executor.evaluate("(and (is-house $0) false)", {0:context})
print(o3)

tensor([1.0353], grad_fn=<MaximumBackward0>)
tensor([-0.1066], grad_fn=<MaximumBackward0>)
tensor([-1.0353], grad_fn=<LogBackward0>) tensor([-0.1066], grad_fn=<LogBackward0>)
tensor([1.0353], grad_fn=<LogBackward0>)
{'end': tensor([-9.2102], grad_fn=<MinimumBackward0>)}


In [5]:
from karanir.utils.tensor import logit
context = {
    "end": logit(torch.tensor([0.00,0.5,1.0])),
    "features": torch.randn([3,obj_dim]),
    "is-red":logit(torch.tensor([0.00,0.5,1.0])),
    "executor": model.central_executor
}
print(context["is-red"])
model.central_executor.apply_action("pick-up", [0,2], context)

print(context["is-red"])


tensor([-9.2102,  0.0000,  9.2101])
(is-red $0) tensor(0.6027, grad_fn=<SigmoidBackward0>)
(if (is-blue $1) (assign (is-blue $0) (true)) (assign (is-red $1) (false)))
tensor([1.8916], grad_fn=<AddBackward0>)


TypeError: Assign() missing 1 required positional argument: 'y'

In [ ]:
-9.2102 * (1 - 0.5361) + 0.5361 * 9.2101

0.6649228300000001